<a href="https://colab.research.google.com/github/mobassir94/comprehensive-bangla-tts/blob/main/mlt_TTS_inference_demo/v1_Multilingual_(ben%2Bara)_tts_based_quranic_tafsir_reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

in this notebook we will be using arabic tacotron2 along with bangla hifi gan vocoder based GlowTTS for multilingual book reading purpose,we will try to read tafsir bayan multilingual (bangla+arabic+english) code mixed tafsir of https://www.hadithbd.com/ with the help of our multilingual TTS pipeline.

special thanks to 
1. [Arabic tacotron2](https://github.com/nipponjo/tts-arabic-tacotron2)
2. [Bangla glowtts](https://github.com/Open-Speech-EkStep/vakyansh-tts)

Based on our previous [notebook](https://github.com/mobassir94/comprehensive-bangla-tts/blob/main/mlt_TTS_inference_demo/bn-ar-tts-pipeline.ipynb)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install gdown==4.5.1
!pip install pydload==1.0.9
!pip install unidecode
!pip install numpy
!pip install pydload==1.0.9
!pip install pydub
!pip install bnunicodenormalizer==0.1.1

# Download and clone necessary files

In [ ]:

import gdown
import torch
import os
import sys
import IPython.display as ipd
import pandas as pd
from pydub import AudioSegment

male = True
git_clone = True

#https://www.kaggle.com/datasets/mobassir/bangla-quran-with-tafsir?select=ben_quran_with_tafsir.csv

q_tafsir = 'https://drive.google.com/drive/folders/1MJnVzKRCLzF_GLRkGALbHcLsHrNBl8xV?usp=sharing'
gdown.download_folder(url=q_tafsir, quiet=True)  

if(git_clone):
  !git clone https://github.com/Open-Speech-EkStep/vakyansh-tts
else:
  url = "https://drive.google.com/drive/folders/14f9LeBKr5HHD2xx44TryjxwJLUyrGpyZ?usp=sharing"
  gdown.download_folder(url=url, quiet=True)  


! git clone https://github.com/nipponjo/tts-arabic-tacotron2
import sys
sys.path.append('/content/tts-arabic-tacotron2')

from model.networks import Tacotron2Wave

#sys.path.append('/content/vakyansh-tts/')
#

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #need cuda

device

Cloning into 'vakyansh-tts'...
remote: Enumerating objects: 1073, done.
remote: Counting objects: 100% (1073/1073), done.
remote: Compressing objects: 100% (411/411), done.
remote: Total 1073 (delta 650), reused 1070 (delta 649), pack-reused 0
Receiving objects: 100% (1073/1073), 402.50 KiB | 17.50 MiB/s, done.
Resolving deltas: 100% (650/650), done.
Cloning into 'tts-arabic-tacotron2'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 58 (delta 3), reused 58 (delta 3), pack-reused 0
Unpacking objects: 100% (58/58), done.


device(type='cuda')

# Create empty audio file of 1 second duration (purpose -> post processing)

In [ ]:
audio = AudioSegment.silent(duration=1000)
sound = audio.set_frame_rate(audio.frame_rate*2)
sound.export("./empty.wav", format="wav")
type(sound)


pydub.audio_segment.AudioSegment

In [ ]:
! echo $PYTHONPATH

/env/python


#Bangla GlowTTS

In [ ]:
%%capture
! pip install bnnumerizer
! pip install bangla==0.0.2
! pip install Unidecode
! pip install numpy


In [ ]:

os.chdir('vakyansh-tts') 

#skipping next 3 line of codes as we have already pytorch latest version installed in this environment

# !bash install.sh
# !python setup.py bdist_wheel
# !pip install -e .

os.chdir('tts_infer')
!mkdir translit_models
os.chdir('translit_models')
!wget https://storage.googleapis.com/vakyaansh-open-models/translit_models/default_lineup.json
!mkdir bangla
os.chdir('bangla')
!wget https://storage.googleapis.com/vakyaansh-open-models/translit_models/bengali/bengali_transliteration.zip
!unzip bengali_transliteration


if(male):
  print("male....")
  !wget https://storage.googleapis.com/vakyansh-open-models/tts/bengali/bn-IN/male_voice_1/glow.zip
  !unzip glow.zip

  !wget https://storage.googleapis.com/vakyansh-open-models/tts/bengali/bn-IN/male_voice_1/hifi.zip
  !unzip hifi.zip
else:
  print("female....")
  !wget https://storage.googleapis.com/vakyansh-open-models/tts/bengali/bn-IN/female_voice_0/glow.zip
  !unzip glow.zip

  !wget https://storage.googleapis.com/vakyansh-open-models/tts/bengali/bn-IN/female_voice_0/hifi.zip
  !unzip hifi.zip

!rm glow.zip
!rm hifi.zip




--2022-10-26 18:25:01--  https://storage.googleapis.com/vakyaansh-open-models/translit_models/default_lineup.json
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3422 (3.3K) [application/json]
Saving to: ‘default_lineup.json’

default_lineup.json 100%[===================>]   3.34K  --.-KB/s    in 0s      

2022-10-26 18:25:01 (35.2 MB/s) - ‘default_lineup.json’ saved [3422/3422]

--2022-10-26 18:25:01--  https://storage.googleapis.com/vakyaansh-open-models/translit_models/bengali/bengali_transliteration.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 404 Not Found


# Removing Annoying print statement during bangla TTS inference

In [ ]:
%%writefile /content/vakyansh-tts/tts_infer/transliterate.py

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import random
import sys
import os
import json
import enum
import traceback
import re

F_DIR = os.path.dirname(os.environ.get('translit_model_base_path', os.path.realpath(__file__)))


class XlitError(enum.Enum):
    lang_err = "Unsupported langauge ID requested ;( Please check available languages."
    string_err = "String passed is incompatable ;("
    internal_err = "Internal crash ;("
    unknown_err = "Unknown Failure"
    loading_err = "Loading failed ;( Check if metadata/paths are correctly configured."


##=================== Network ==================================================


class Encoder(nn.Module):
    def __init__(
        self,
        input_dim,
        embed_dim,
        hidden_dim,
        rnn_type="gru",
        layers=1,
        bidirectional=False,
        dropout=0,
        device="cpu",
    ):
        super(Encoder, self).__init__()

        self.input_dim = input_dim  # src_vocab_sz
        self.enc_embed_dim = embed_dim
        self.enc_hidden_dim = hidden_dim
        self.enc_rnn_type = rnn_type
        self.enc_layers = layers
        self.enc_directions = 2 if bidirectional else 1
        self.device = device

        self.embedding = nn.Embedding(self.input_dim, self.enc_embed_dim)

        if self.enc_rnn_type == "gru":
            self.enc_rnn = nn.GRU(
                input_size=self.enc_embed_dim,
                hidden_size=self.enc_hidden_dim,
                num_layers=self.enc_layers,
                bidirectional=bidirectional,
            )
        elif self.enc_rnn_type == "lstm":
            self.enc_rnn = nn.LSTM(
                input_size=self.enc_embed_dim,
                hidden_size=self.enc_hidden_dim,
                num_layers=self.enc_layers,
                bidirectional=bidirectional,
            )
        else:
            raise Exception("XlitError: unknown RNN type mentioned")

    def forward(self, x, x_sz, hidden=None):
        """
        x_sz: (batch_size, 1) -  Unpadded sequence lengths used for pack_pad
        """
        batch_sz = x.shape[0]
        # x: batch_size, max_length, enc_embed_dim
        x = self.embedding(x)

        ## pack the padded data
        # x: max_length, batch_size, enc_embed_dim -> for pack_pad
        x = x.permute(1, 0, 2)
        x = nn.utils.rnn.pack_padded_sequence(x, x_sz, enforce_sorted=False)  # unpad

        # output: packed_size, batch_size, enc_embed_dim
        # hidden: n_layer**num_directions, batch_size, hidden_dim | if LSTM (h_n, c_n)
        output, hidden = self.enc_rnn(
            x
        )  # gru returns hidden state of all timesteps as well as hidden state at last timestep

        ## pad the sequence to the max length in the batch
        # output: max_length, batch_size, enc_emb_dim*directions)
        output, _ = nn.utils.rnn.pad_packed_sequence(output)

        # output: batch_size, max_length, hidden_dim
        output = output.permute(1, 0, 2)

        return output, hidden

    def get_word_embedding(self, x):
        """ """
        x_sz = torch.tensor([len(x)])
        x_ = torch.tensor(x).unsqueeze(0).to(dtype=torch.long)
        # x: 1, max_length, enc_embed_dim
        x = self.embedding(x_)

        ## pack the padded data
        # x: max_length, 1, enc_embed_dim -> for pack_pad
        x = x.permute(1, 0, 2)
        x = nn.utils.rnn.pack_padded_sequence(x, x_sz, enforce_sorted=False)  # unpad

        # output: packed_size, 1, enc_embed_dim
        # hidden: n_layer**num_directions, 1, hidden_dim | if LSTM (h_n, c_n)
        output, hidden = self.enc_rnn(
            x
        )  # gru returns hidden state of all timesteps as well as hidden state at last timestep

        out_embed = hidden[0].squeeze()

        return out_embed


class Decoder(nn.Module):
    def __init__(
        self,
        output_dim,
        embed_dim,
        hidden_dim,
        rnn_type="gru",
        layers=1,
        use_attention=True,
        enc_outstate_dim=None,  # enc_directions * enc_hidden_dim
        dropout=0,
        device="cpu",
    ):
        super(Decoder, self).__init__()

        self.output_dim = output_dim  # tgt_vocab_sz
        self.dec_hidden_dim = hidden_dim
        self.dec_embed_dim = embed_dim
        self.dec_rnn_type = rnn_type
        self.dec_layers = layers
        self.use_attention = use_attention
        self.device = device
        if self.use_attention:
            self.enc_outstate_dim = enc_outstate_dim if enc_outstate_dim else hidden_dim
        else:
            self.enc_outstate_dim = 0

        self.embedding = nn.Embedding(self.output_dim, self.dec_embed_dim)

        if self.dec_rnn_type == "gru":
            self.dec_rnn = nn.GRU(
                input_size=self.dec_embed_dim
                + self.enc_outstate_dim,  # to concat attention_output
                hidden_size=self.dec_hidden_dim,  # previous Hidden
                num_layers=self.dec_layers,
                batch_first=True,
            )
        elif self.dec_rnn_type == "lstm":
            self.dec_rnn = nn.LSTM(
                input_size=self.dec_embed_dim
                + self.enc_outstate_dim,  # to concat attention_output
                hidden_size=self.dec_hidden_dim,  # previous Hidden
                num_layers=self.dec_layers,
                batch_first=True,
            )
        else:
            raise Exception("XlitError: unknown RNN type mentioned")

        self.fc = nn.Sequential(
            nn.Linear(self.dec_hidden_dim, self.dec_embed_dim),
            nn.LeakyReLU(),
            # nn.Linear(self.dec_embed_dim, self.dec_embed_dim), nn.LeakyReLU(), # removing to reduce size
            nn.Linear(self.dec_embed_dim, self.output_dim),
        )

        ##----- Attention ----------
        if self.use_attention:
            self.W1 = nn.Linear(self.enc_outstate_dim, self.dec_hidden_dim)
            self.W2 = nn.Linear(self.dec_hidden_dim, self.dec_hidden_dim)
            self.V = nn.Linear(self.dec_hidden_dim, 1)

    def attention(self, x, hidden, enc_output):
        """
        x: (batch_size, 1, dec_embed_dim) -> after Embedding
        enc_output: batch_size, max_length, enc_hidden_dim *num_directions
        hidden: n_layers, batch_size, hidden_size | if LSTM (h_n, c_n)
        """

        ## perform addition to calculate the score

        # hidden_with_time_axis: batch_size, 1, hidden_dim
        ## hidden_with_time_axis = hidden.permute(1, 0, 2) ## replaced with below 2lines
        hidden_with_time_axis = (
            torch.sum(hidden, axis=0)
            if self.dec_rnn_type != "lstm"
            else torch.sum(hidden[0], axis=0)
        )  # h_n

        hidden_with_time_axis = hidden_with_time_axis.unsqueeze(1)

        # score: batch_size, max_length, hidden_dim
        score = torch.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis))

        # attention_weights: batch_size, max_length, 1
        # we get 1 at the last axis because we are applying score to self.V
        attention_weights = torch.softmax(self.V(score), dim=1)

        # context_vector shape after sum == (batch_size, hidden_dim)
        context_vector = attention_weights * enc_output
        context_vector = torch.sum(context_vector, dim=1)
        # context_vector: batch_size, 1, hidden_dim
        context_vector = context_vector.unsqueeze(1)

        # attend_out (batch_size, 1, dec_embed_dim + hidden_size)
        attend_out = torch.cat((context_vector, x), -1)

        return attend_out, attention_weights

    def forward(self, x, hidden, enc_output):
        """
        x: (batch_size, 1)
        enc_output: batch_size, max_length, dec_embed_dim
        hidden: n_layer, batch_size, hidden_size | lstm: (h_n, c_n)
        """
        if (hidden is None) and (self.use_attention is False):
            raise Exception(
                "XlitError: No use of a decoder with No attention and No Hidden"
            )

        batch_sz = x.shape[0]

        if hidden is None:
            # hidden: n_layers, batch_size, hidden_dim
            hid_for_att = torch.zeros(
                (self.dec_layers, batch_sz, self.dec_hidden_dim)
            ).to(self.device)
        elif self.dec_rnn_type == "lstm":
            hid_for_att = hidden[1]  # c_n

        # x (batch_size, 1, dec_embed_dim) -> after embedding
        x = self.embedding(x)

        if self.use_attention:
            # x (batch_size, 1, dec_embed_dim + hidden_size) -> after attention
            # aw: (batch_size, max_length, 1)
            x, aw = self.attention(x, hidden, enc_output)
        else:
            x, aw = x, 0

        # passing the concatenated vector to the GRU
        # output: (batch_size, n_layers, hidden_size)
        # hidden: n_layers, batch_size, hidden_size | if LSTM (h_n, c_n)
        output, hidden = (
            self.dec_rnn(x, hidden) if hidden is not None else self.dec_rnn(x)
        )

        # output :shp: (batch_size * 1, hidden_size)
        output = output.view(-1, output.size(2))

        # output :shp: (batch_size * 1, output_dim)
        output = self.fc(output)

        return output, hidden, aw


class Seq2Seq(nn.Module):
    """
    Class dependency: Encoder, Decoder
    """

    def __init__(
        self, encoder, decoder, pass_enc2dec_hid=False, dropout=0, device="cpu"
    ):
        super(Seq2Seq, self).__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.pass_enc2dec_hid = pass_enc2dec_hid
        _force_en2dec_hid_conv = False

        if self.pass_enc2dec_hid:
            assert (
                decoder.dec_hidden_dim == encoder.enc_hidden_dim
            ), "Hidden Dimension of encoder and decoder must be same, or unset `pass_enc2dec_hid`"
        if decoder.use_attention:
            assert (
                decoder.enc_outstate_dim
                == encoder.enc_directions * encoder.enc_hidden_dim
            ), "Set `enc_out_dim` correctly in decoder"
        assert (
            self.pass_enc2dec_hid or decoder.use_attention
        ), "No use of a decoder with No attention and No Hidden from Encoder"

        self.use_conv_4_enc2dec_hid = False
        if (
            self.pass_enc2dec_hid
            and (encoder.enc_directions * encoder.enc_layers != decoder.dec_layers)
        ) or _force_en2dec_hid_conv:
            if encoder.enc_rnn_type == "lstm" or encoder.enc_rnn_type == "lstm":
                raise Exception(
                    "XlitError: conv for enc2dec_hid not implemented; Change the layer numbers appropriately"
                )

            self.use_conv_4_enc2dec_hid = True
            self.enc_hid_1ax = encoder.enc_directions * encoder.enc_layers
            self.dec_hid_1ax = decoder.dec_layers
            self.e2d_hidden_conv = nn.Conv1d(self.enc_hid_1ax, self.dec_hid_1ax, 1)

    def enc2dec_hidden(self, enc_hidden):
        """
        enc_hidden: n_layer, batch_size, hidden_dim*num_directions
        TODO: Implement the logic for LSTm bsed model
        """
        # hidden: batch_size, enc_layer*num_directions, enc_hidden_dim
        hidden = enc_hidden.permute(1, 0, 2).contiguous()
        # hidden: batch_size, dec_layers, dec_hidden_dim -> [N,C,Tstep]
        hidden = self.e2d_hidden_conv(hidden)

        # hidden: dec_layers, batch_size , dec_hidden_dim
        hidden_for_dec = hidden.permute(1, 0, 2).contiguous()

        return hidden_for_dec

    def active_beam_inference(self, src, beam_width=3, max_tgt_sz=50):
        """Search based decoding
        src: (sequence_len)
        """

        def _avg_score(p_tup):
            """Used for Sorting
            TODO: Dividing by length of sequence power alpha as hyperparam
            """
            return p_tup[0]

        import sys

        batch_size = 1
        start_tok = src[0]
        end_tok = src[-1]
        src_sz = torch.tensor([len(src)])
        src_ = src.unsqueeze(0)

        # enc_output: (batch_size, padded_seq_length, enc_hidden_dim*num_direction)
        # enc_hidden: (enc_layers*num_direction, batch_size, hidden_dim)
        enc_output, enc_hidden = self.encoder(src_, src_sz)

        if self.pass_enc2dec_hid:
            # dec_hidden: dec_layers, batch_size , dec_hidden_dim
            if self.use_conv_4_enc2dec_hid:
                init_dec_hidden = self.enc2dec_hidden(enc_hidden)
            else:
                init_dec_hidden = enc_hidden
        else:
            # dec_hidden -> Will be initialized to zeros internally
            init_dec_hidden = None

        # top_pred[][0] = Σ-log_softmax
        # top_pred[][1] = sequence torch.tensor shape: (1)
        # top_pred[][2] = dec_hidden
        top_pred_list = [(0, start_tok.unsqueeze(0), init_dec_hidden)]

        for t in range(max_tgt_sz):
            cur_pred_list = []

            for p_tup in top_pred_list:
                if p_tup[1][-1] == end_tok:
                    cur_pred_list.append(p_tup)
                    continue

                # dec_hidden: dec_layers, 1, hidden_dim
                # dec_output: 1, output_dim
                dec_output, dec_hidden, _ = self.decoder(
                    x=p_tup[1][-1].view(1, 1),  # dec_input: (1,1)
                    hidden=p_tup[2],
                    enc_output=enc_output,
                )

                ## π{prob} = Σ{log(prob)} -> to prevent diminishing
                # dec_output: (1, output_dim)
                dec_output = nn.functional.log_softmax(dec_output, dim=1)
                # pred_topk.values & pred_topk.indices: (1, beam_width)
                pred_topk = torch.topk(dec_output, k=beam_width, dim=1)

                for i in range(beam_width):
                    sig_logsmx_ = p_tup[0] + pred_topk.values[0][i]
                    # seq_tensor_ : (seq_len)
                    seq_tensor_ = torch.cat((p_tup[1], pred_topk.indices[0][i].view(1)))

                    cur_pred_list.append((sig_logsmx_, seq_tensor_, dec_hidden))

            cur_pred_list.sort(key=_avg_score, reverse=True)  # Maximized order
            top_pred_list = cur_pred_list[:beam_width]

            # check if end_tok of all topk
            end_flags_ = [1 if t[1][-1] == end_tok else 0 for t in top_pred_list]
            if beam_width == sum(end_flags_):
                break

        pred_tnsr_list = [t[1] for t in top_pred_list]

        return pred_tnsr_list


##===================== Glyph handlers =======================================


class GlyphStrawboss:
    def __init__(self, glyphs="en"):
        """list of letters in a language in unicode
        lang: ISO Language code
        glyphs: json file with script information
        """
        if glyphs == "en":
            # Smallcase alone
            self.glyphs = [chr(alpha) for alpha in range(97, 122 + 1)]
        else:
            self.dossier = json.load(open(glyphs, encoding="utf-8"))
            self.glyphs = self.dossier["glyphs"]
            self.numsym_map = self.dossier["numsym_map"]

        self.char2idx = {}
        self.idx2char = {}
        self._create_index()

    def _create_index(self):

        self.char2idx["_"] = 0  # pad
        self.char2idx["$"] = 1  # start
        self.char2idx["#"] = 2  # end
        self.char2idx["*"] = 3  # Mask
        self.char2idx["'"] = 4  # apostrophe U+0027
        self.char2idx["%"] = 5  # unused
        self.char2idx["!"] = 6  # unused

        # letter to index mapping
        for idx, char in enumerate(self.glyphs):
            self.char2idx[char] = idx + 7  # +7 token initially

        # index to letter mapping
        for char, idx in self.char2idx.items():
            self.idx2char[idx] = char

    def size(self):
        return len(self.char2idx)

    def word2xlitvec(self, word):
        """Converts given string of gyphs(word) to vector(numpy)
        Also adds tokens for start and end
        """
        try:
            vec = [self.char2idx["$"]]  # start token
            for i in list(word):
                vec.append(self.char2idx[i])
            vec.append(self.char2idx["#"])  # end token

            vec = np.asarray(vec, dtype=np.int64)
            return vec

        except Exception as error:
            print("XlitError: In word:", word, "Error Char not in Token:", error)
            sys.exit()

    def xlitvec2word(self, vector):
        """Converts vector(numpy) to string of glyphs(word)"""
        char_list = []
        for i in vector:
            char_list.append(self.idx2char[i])

        word = "".join(char_list).replace("$", "").replace("#", "")  # remove tokens
        word = word.replace("_", "").replace("*", "")  # remove tokens
        return word


class VocabSanitizer:
    def __init__(self, data_file):
        """
        data_file: path to file conatining vocabulary list
        """
        extension = os.path.splitext(data_file)[-1]
        if extension == ".json":
            self.vocab_set = set(json.load(open(data_file, encoding="utf-8")))
        elif extension == ".csv":
            self.vocab_df = pd.read_csv(data_file).set_index("WORD")
            self.vocab_set = set(self.vocab_df.index)
        else:
            print("XlitError: Only Json/CSV file extension supported")

    def reposition(self, word_list):
        """Reorder Words in list"""
        new_list = []
        temp_ = word_list.copy()
        for v in word_list:
            if v in self.vocab_set:
                new_list.append(v)
                temp_.remove(v)
        new_list.extend(temp_)

        return new_list


##=============== INSTANTIATION ================================================


class XlitPiston:
    """
    For handling prediction & post-processing of transliteration for a single language
    Class dependency: Seq2Seq, GlyphStrawboss, VocabSanitizer
    Global Variables: F_DIR
    """

    def __init__(
        self,
        weight_path,
        vocab_file,
        tglyph_cfg_file,
        iglyph_cfg_file="en",
        device="cpu",
    ):

        self.device = device
        self.in_glyph_obj = GlyphStrawboss(iglyph_cfg_file)
        self.tgt_glyph_obj = GlyphStrawboss(glyphs=tglyph_cfg_file)
        self.voc_sanity = VocabSanitizer(vocab_file)

        self._numsym_set = set(
            json.load(open(tglyph_cfg_file, encoding="utf-8"))["numsym_map"].keys()
        )
        self._inchar_set = set("abcdefghijklmnopqrstuvwxyz")
        self._natscr_set = set().union(
            self.tgt_glyph_obj.glyphs, sum(self.tgt_glyph_obj.numsym_map.values(), [])
        )

        ## Model Config Static                TODO: add defining in json support
        input_dim = self.in_glyph_obj.size()
        output_dim = self.tgt_glyph_obj.size()
        enc_emb_dim = 300
        dec_emb_dim = 300
        enc_hidden_dim = 512
        dec_hidden_dim = 512
        rnn_type = "lstm"
        enc2dec_hid = True
        attention = True
        enc_layers = 1
        dec_layers = 2
        m_dropout = 0
        enc_bidirect = True
        enc_outstate_dim = enc_hidden_dim * (2 if enc_bidirect else 1)

        enc = Encoder(
            input_dim=input_dim,
            embed_dim=enc_emb_dim,
            hidden_dim=enc_hidden_dim,
            rnn_type=rnn_type,
            layers=enc_layers,
            dropout=m_dropout,
            device=self.device,
            bidirectional=enc_bidirect,
        )
        dec = Decoder(
            output_dim=output_dim,
            embed_dim=dec_emb_dim,
            hidden_dim=dec_hidden_dim,
            rnn_type=rnn_type,
            layers=dec_layers,
            dropout=m_dropout,
            use_attention=attention,
            enc_outstate_dim=enc_outstate_dim,
            device=self.device,
        )
        self.model = Seq2Seq(enc, dec, pass_enc2dec_hid=enc2dec_hid, device=self.device)
        self.model = self.model.to(self.device)
        weights = torch.load(weight_path, map_location=torch.device(self.device))

        self.model.load_state_dict(weights)
        self.model.eval()

    def character_model(self, word, beam_width=1):
        in_vec = torch.from_numpy(self.in_glyph_obj.word2xlitvec(word)).to(self.device)
        ## change to active or passive beam
        p_out_list = self.model.active_beam_inference(in_vec, beam_width=beam_width)
        p_result = [
            self.tgt_glyph_obj.xlitvec2word(out.cpu().numpy()) for out in p_out_list
        ]

        result = self.voc_sanity.reposition(p_result)

        # List type
        return result

    def numsym_model(self, seg):
        """tgt_glyph_obj.numsym_map[x] returns a list object"""
        if len(seg) == 1:
            return [seg] + self.tgt_glyph_obj.numsym_map[seg]

        a = [self.tgt_glyph_obj.numsym_map[n][0] for n in seg]
        return [seg] + ["".join(a)]

    def _word_segementer(self, sequence):

        sequence = sequence.lower()
        accepted = set().union(self._numsym_set, self._inchar_set, self._natscr_set)
        # sequence = ''.join([i for i in sequence if i in accepted])

        segment = []
        idx = 0
        seq_ = list(sequence)
        while len(seq_):
            # for Number-Symbol
            temp = ""
            while len(seq_) and seq_[0] in self._numsym_set:
                temp += seq_[0]
                seq_.pop(0)
            if temp != "":
                segment.append(temp)

            # for Target Chars
            temp = ""
            while len(seq_) and seq_[0] in self._natscr_set:
                temp += seq_[0]
                seq_.pop(0)
            if temp != "":
                segment.append(temp)

            # for Input-Roman Chars
            temp = ""
            while len(seq_) and seq_[0] in self._inchar_set:
                temp += seq_[0]
                seq_.pop(0)
            if temp != "":
                segment.append(temp)

            temp = ""
            while len(seq_) and seq_[0] not in accepted:
                temp += seq_[0]
                seq_.pop(0)
            if temp != "":
                segment.append(temp)

        return segment

    def inferencer(self, sequence, beam_width=10):

        seg = self._word_segementer(sequence[:120])
        lit_seg = []

        p = 0
        while p < len(seg):
            if seg[p][0] in self._natscr_set:
                lit_seg.append([seg[p]])
                p += 1

            elif seg[p][0] in self._inchar_set:
                lit_seg.append(self.character_model(seg[p], beam_width=beam_width))
                p += 1

            elif seg[p][0] in self._numsym_set:  # num & punc
                lit_seg.append(self.numsym_model(seg[p]))
                p += 1
            else:
                lit_seg.append([seg[p]])
                p += 1

        ## IF segment less/equal to 2 then return combinotorial,
        ## ELSE only return top1 of each result concatenated
        if len(lit_seg) == 1:
            final_result = lit_seg[0]

        elif len(lit_seg) == 2:
            final_result = [""]
            for seg in lit_seg:
                new_result = []
                for s in seg:
                    for f in final_result:
                        new_result.append(f + s)
                final_result = new_result

        else:
            new_result = []
            for seg in lit_seg:
                new_result.append(seg[0])
            final_result = ["".join(new_result)]

        return final_result


from collections.abc import Iterable
from pydload import dload
import zipfile

MODEL_DOWNLOAD_URL_PREFIX = "https://github.com/AI4Bharat/IndianNLP-Transliteration/releases/download/xlit_v0.5.0/"


def is_folder_writable(folder):
    try:
        os.makedirs(folder, exist_ok=True)
        tmp_file = os.path.join(folder, ".write_test")
        with open(tmp_file, "w") as f:
            f.write("Permission Check")
        os.remove(tmp_file)
        return True
    except:
        return False


def is_directory_writable(path):
    if os.name == "nt":
        return is_folder_writable(path)
    return os.access(path, os.W_OK | os.X_OK)


class XlitEngine:
    """
    For Managing the top level tasks and applications of transliteration
    Global Variables: F_DIR
    """

    def __init__(
        self, lang2use="all", config_path="translit_models/default_lineup.json"
    ):

        lineup = json.load(open(os.path.join(F_DIR, config_path), encoding="utf-8"))
        self.lang_config = {}
        if isinstance(lang2use, str):
            if lang2use == "all":
                self.lang_config = lineup
            elif lang2use in lineup:
                self.lang_config[lang2use] = lineup[lang2use]
            else:
                raise Exception(
                    "XlitError: The entered Langauge code not found. Available are {}".format(
                        lineup.keys()
                    )
                )

        elif isinstance(lang2use, Iterable):
            for l in lang2use:
                try:
                    self.lang_config[l] = lineup[l]
                except:
                    print(
                        "XlitError: Language code {} not found, Skipping...".format(l)
                    )
        else:
            raise Exception(
                "XlitError: lang2use must be a list of language codes (or) string of single language code"
            )

        if is_directory_writable(F_DIR):
            models_path = os.path.join(F_DIR, "translit_models")
        else:
            user_home = os.path.expanduser("~")
            models_path = os.path.join(user_home, ".AI4Bharat_Xlit_Models")
        os.makedirs(models_path, exist_ok=True)
        self.download_models(models_path)

        self.langs = {}
        self.lang_model = {}
        for la in self.lang_config:
            try:
                #print("Loading {}...".format(la))
                self.lang_model[la] = XlitPiston(
                    weight_path=os.path.join(
                        models_path, self.lang_config[la]["weight"]
                    ),
                    vocab_file=os.path.join(models_path, self.lang_config[la]["vocab"]),
                    tglyph_cfg_file=os.path.join(
                        models_path, self.lang_config[la]["script"]
                    ),
                    iglyph_cfg_file="en",
                )
                self.langs[la] = self.lang_config[la]["name"]
            except Exception as error:
                print("XlitError: Failure in loading {} \n".format(la), error)
                print(XlitError.loading_err.value)

    def download_models(self, models_path):
        """
        Download models from GitHub Releases if not exists
        """
        for l in self.lang_config:
            lang_name = self.lang_config[l]["eng_name"]
            lang_model_path = os.path.join(models_path, lang_name)
            if not os.path.isdir(lang_model_path):
                print("Downloading model for language: %s" % lang_name)
                remote_url = MODEL_DOWNLOAD_URL_PREFIX + lang_name + ".zip"
                downloaded_zip_path = os.path.join(models_path, lang_name + ".zip")
                dload(url=remote_url, save_to_path=downloaded_zip_path, max_time=None)

                if not os.path.isfile(downloaded_zip_path):
                    exit(
                        f"ERROR: Unable to download model from {remote_url} into {models_path}"
                    )

                with zipfile.ZipFile(downloaded_zip_path, "r") as zip_ref:
                    zip_ref.extractall(models_path)

                if os.path.isdir(lang_model_path):
                    os.remove(downloaded_zip_path)
                else:
                    exit(
                        f"ERROR: Unable to find models in {lang_model_path} after download"
                    )
        return

    def translit_word(self, eng_word, lang_code="default", topk=7, beam_width=10):
        if eng_word == "":
            return []

        if lang_code in self.langs:
            try:
                res_list = self.lang_model[lang_code].inferencer(
                    eng_word, beam_width=beam_width
                )
                return res_list[:topk]

            except Exception as error:
                print("XlitError:", traceback.format_exc())
                print(XlitError.internal_err.value)
                return XlitError.internal_err

        elif lang_code == "default":
            try:
                res_dict = {}
                for la in self.lang_model:
                    res = self.lang_model[la].inferencer(
                        eng_word, beam_width=beam_width
                    )
                    res_dict[la] = res[:topk]
                return res_dict

            except Exception as error:
                print("XlitError:", traceback.format_exc())
                print(XlitError.internal_err.value)
                return XlitError.internal_err

        else:
            print("XlitError: Unknown Langauge requested", lang_code)
            print(XlitError.lang_err.value)
            return XlitError.lang_err

    def translit_sentence(self, eng_sentence, lang_code="default", beam_width=10):
        if eng_sentence == "":
            return []

        if lang_code in self.langs:
            try:
                out_str = ""
                for word in eng_sentence.split():
                    res_ = self.lang_model[lang_code].inferencer(
                        word, beam_width=beam_width
                    )
                    out_str = out_str + res_[0] + " "
                return out_str[:-1]

            except Exception as error:
                print("XlitError:", traceback.format_exc())
                print(XlitError.internal_err.value)
                return XlitError.internal_err

        elif lang_code == "default":
            try:
                res_dict = {}
                for la in self.lang_model:
                    out_str = ""
                    for word in eng_sentence.split():
                        res_ = self.lang_model[la].inferencer(
                            word, beam_width=beam_width
                        )
                        out_str = out_str + res_[0] + " "
                    res_dict[la] = out_str[:-1]
                return res_dict

            except Exception as error:
                print("XlitError:", traceback.format_exc())
                print(XlitError.internal_err.value)
                return XlitError.internal_err

        else:
            print("XlitError: Unknown Langauge requested", lang_code)
            print(XlitError.lang_err.value)
            return XlitError.lang_err


if __name__ == "__main__":

    available_lang = [
        "bn",
        "gu",
        "hi",
        "kn",
        "gom",
        "mai",
        "ml",
        "mr",
        "pa",
        "sd",
        "si",
        "ta",
        "te",
        "ur",
    ]

    reg = re.compile(r"[a-zA-Z]")
    lang = "hi"
    engine = XlitEngine(
        lang
    )  # if you don't specify lang code here, this will give results in all langs available
    sent = "Hello World! ABCD क्या हाल है आपका?"
    words = [
        engine.translit_word(word, topk=1)[lang][0] if reg.match(word) else word
        for word in sent.split()
    ]  # only transliterated en words, leaves rest as it is
    updated_sent = " ".join(words)

    print(updated_sent)

    # output : हेलो वर्ल्ड! क्या हाल है आपका?

    # y = engine.translit_sentence("Hello World !")['hi']
    # print(y)


Overwriting /content/vakyansh-tts/tts_infer/transliterate.py


# Fixing arabic tts and bangla tts module collision

both arabic and bangla tts has common module name "text". as a result we couldn't do 'from tts_infer.tts import TextToMel, MelToWav' after importing modules from arabic tts package,because the system seeks TextToMel and MelToWav from arabic tts directory instead.to fix this issue we change the bangla tts inference script below so that we can directly import TextToMel and  MelToWav class.


"treat the code cell below as complete black box for you"

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
from typing import Tuple
import sys
from argparse import ArgumentParser
sys.path.append('/content/vakyansh-tts/src/glow_tts')
import torch
import numpy as np
import os
import json


from scipy.io.wavfile import write
from hifi.env import AttrDict
from hifi.models import Generator


#from text import text_to_sequence
import commons
import models
import os
import glob
import sys
import argparse
import logging
import json
import subprocess
import numpy as np
from scipy.io.wavfile import read
import torch

MATPLOTLIB_FLAG = False

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging


def load_checkpoint(checkpoint_path, model, optimizer=None):
    assert os.path.isfile(checkpoint_path)
    checkpoint_dict = torch.load(checkpoint_path, map_location="cpu")
    iteration = 1
    if "iteration" in checkpoint_dict.keys():
        iteration = checkpoint_dict["iteration"]
    if "learning_rate" in checkpoint_dict.keys():
        learning_rate = checkpoint_dict["learning_rate"]
    if optimizer is not None and "optimizer" in checkpoint_dict.keys():
        optimizer.load_state_dict(checkpoint_dict["optimizer"])
    saved_state_dict = checkpoint_dict["model"]
    if hasattr(model, "module"):
        state_dict = model.module.state_dict()
    else:
        state_dict = model.state_dict()
    new_state_dict = {}
    for k, v in state_dict.items():
        try:
            new_state_dict[k] = saved_state_dict[k]
        except:
            logger.info("%s is not in the checkpoint" % k)
            new_state_dict[k] = v
    if hasattr(model, "module"):
        model.module.load_state_dict(new_state_dict)
    else:
        model.load_state_dict(new_state_dict)
    logger.info(
        "Loaded checkpoint '{}' (iteration {})".format(checkpoint_path, iteration)
    )
    return model, optimizer, learning_rate, iteration


def save_checkpoint(model, optimizer, learning_rate, iteration, checkpoint_path):
    logger.info(
        "Saving model and optimizer state at iteration {} to {}".format(
            iteration, checkpoint_path
        )
    )
    if hasattr(model, "module"):
        state_dict = model.module.state_dict()
    else:
        state_dict = model.state_dict()
    torch.save(
        {
            "model": state_dict,
            "iteration": iteration,
            "optimizer": optimizer.state_dict(),
            "learning_rate": learning_rate,
        },
        checkpoint_path,
    )


def summarize(writer, global_step, scalars={}, histograms={}, images={}):
    for k, v in scalars.items():
        writer.add_scalar(k, v, global_step)
    for k, v in histograms.items():
        writer.add_histogram(k, v, global_step)
    for k, v in images.items():
        writer.add_image(k, v, global_step, dataformats="HWC")


def latest_checkpoint_path(dir_path, regex="G_*.pth"):
    f_list = glob.glob(os.path.join(dir_path, regex))
    f_list.sort(key=lambda f: int("".join(filter(str.isdigit, f))))
    x = f_list[-1]
    print(x)
    return x


def plot_spectrogram_to_numpy(spectrogram):
    global MATPLOTLIB_FLAG
    if not MATPLOTLIB_FLAG:
        import matplotlib

        matplotlib.use("Agg")
        MATPLOTLIB_FLAG = True
        mpl_logger = logging.getLogger("matplotlib")
        mpl_logger.setLevel(logging.WARNING)
    import matplotlib.pylab as plt
    import numpy as np

    fig, ax = plt.subplots()
    im = ax.imshow(spectrogram, aspect="auto", origin="lower", interpolation="none")
    plt.colorbar(im, ax=ax)
    plt.xlabel("Frames")
    plt.ylabel("Channels")
    plt.tight_layout()

    fig.canvas.draw()
    data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep="")
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close()
    return data


def plot_alignment_to_numpy(alignment, info=None):
    global MATPLOTLIB_FLAG
    if not MATPLOTLIB_FLAG:
        import matplotlib

        matplotlib.use("Agg")
        MATPLOTLIB_FLAG = True
        mpl_logger = logging.getLogger("matplotlib")
        mpl_logger.setLevel(logging.WARNING)
    import matplotlib.pylab as plt
    import numpy as np

    fig, ax = plt.subplots(figsize=(6, 4))
    im = ax.imshow(alignment, aspect="auto", origin="lower", interpolation="none")
    fig.colorbar(im, ax=ax)
    xlabel = "Decoder timestep"
    if info is not None:
        xlabel += "\n\n" + info
    plt.xlabel(xlabel)
    plt.ylabel("Encoder timestep")
    plt.tight_layout()

    fig.canvas.draw()
    data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep="")
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close()
    return data


def load_wav_to_torch(full_path):
    sampling_rate, data = read(full_path)
    return torch.FloatTensor(data.astype(np.float32)), sampling_rate


def load_filepaths_and_text(filename, split="|"):
    with open(filename, encoding="utf-8") as f:
        filepaths_and_text = [line.strip().split(split) for line in f]
    return filepaths_and_text


def get_hparams(init=True):
    parser = argparse.ArgumentParser()
    parser.add_argument("-c", "--config", type=str, help="JSON file for configuration")
    parser.add_argument("-m", "--model", type=str, help="Model name")
    # parser.add_argument('-g', '--gan', type=str,
    #                     help='Model name')
    parser.add_argument("-l", "--logs", type=str, help="logs name")
    # parser.add_argument('-s', '--mels', type=str,
    #                     help='logs name')

    args = parser.parse_args()
    # model_dir = os.path.join("./logs", args.model)
    model_dir = args.model
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    config_path = args.config
    config_save_path = os.path.join(model_dir, "config.json")

    # if not config_path : config_path = config_save_path

    if init:
        with open(config_path, "r") as f:
            data = f.read()
        with open(config_save_path, "w") as f:
            f.write(data)
    else:
        with open(config_save_path, "r") as f:
            data = f.read()
    config = json.loads(data)

    hparams = HParams(**config)
    hparams.model_dir = model_dir
    hparams.log_dir = args.logs
    # hparams.mels_dir = args.mels
    # hparams.gan_dir = args.gan
    return hparams


def get_hparams_from_dir(model_dir):
    config_save_path = os.path.join(model_dir, "config.json")
    with open(config_save_path, "r") as f:
        data = f.read()
    config = json.loads(data)

    hparams = HParams(**config)
    hparams.model_dir = model_dir
    return hparams


def get_hparams_from_file(config_path):
    with open(config_path, "r") as f:
        data = f.read()
    config = json.loads(data)

    hparams = HParams(**config)
    return hparams


def check_git_hash(model_dir):
    source_dir = os.path.dirname(os.path.realpath(__file__))
    if not os.path.exists(os.path.join(source_dir, ".git")):
        logger.warn(
            "{} is not a git repository, therefore hash value comparison will be ignored.".format(
                source_dir
            )
        )
        return

    cur_hash = subprocess.getoutput("git rev-parse HEAD")

    path = os.path.join(model_dir, "githash")
    if os.path.exists(path):
        saved_hash = open(path).read()
        if saved_hash != cur_hash:
            logger.warn(
                "git hash values are different. {}(saved) != {}(current)".format(
                    saved_hash[:8], cur_hash[:8]
                )
            )
    else:
        open(path, "w").write(cur_hash)


def get_logger(model_dir, filename="train.log"):
    global logger
    logger = logging.getLogger(os.path.basename(model_dir))
    logger.setLevel(logging.DEBUG)

    formatter = logging.Formatter("%(asctime)s\t%(name)s\t%(levelname)s\t%(message)s")
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    h = logging.FileHandler(os.path.join(model_dir, filename))
    h.setLevel(logging.DEBUG)
    h.setFormatter(formatter)
    logger.addHandler(h)
    return logger


class HParams:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            if type(v) == dict:
                v = HParams(**v)
            self[k] = v

    def keys(self):
        return self.__dict__.keys()

    def items(self):
        return self.__dict__.items()

    def values(self):
        return self.__dict__.values()

    def __len__(self):
        return len(self.__dict__)

    def __getitem__(self, key):
        return getattr(self, key)

    def __setitem__(self, key, value):
        return setattr(self, key, value)

    def __contains__(self, key):
        return key in self.__dict__

    def __repr__(self):
        return self.__dict__.__repr__()


""" from https://github.com/keithito/tacotron """
import re


from unidecode import unidecode
import inflect


_inflect = inflect.engine()
_comma_number_re = re.compile(r'([0-9][0-9\,]+[0-9])')
_decimal_number_re = re.compile(r'([0-9]+\.[0-9]+)')
_pounds_re = re.compile(r'£([0-9\,]*[0-9]+)')
_dollars_re = re.compile(r'\$([0-9\.\,]*[0-9]+)')
_ordinal_re = re.compile(r'[0-9]+(st|nd|rd|th)')
_number_re = re.compile(r'[0-9]+')

cleaner_names = ['basic_indic_cleaners']

def _remove_commas(m):
  return m.group(1).replace(',', '')


def _expand_decimal_point(m):
  return m.group(1).replace('.', ' point ')


def _expand_dollars(m):
  match = m.group(1)
  parts = match.split('.')
  if len(parts) > 2:
    return match + ' dollars'  # Unexpected format
  dollars = int(parts[0]) if parts[0] else 0
  cents = int(parts[1]) if len(parts) > 1 and parts[1] else 0
  if dollars and cents:
    dollar_unit = 'dollar' if dollars == 1 else 'dollars'
    cent_unit = 'cent' if cents == 1 else 'cents'
    return '%s %s, %s %s' % (dollars, dollar_unit, cents, cent_unit)
  elif dollars:
    dollar_unit = 'dollar' if dollars == 1 else 'dollars'
    return '%s %s' % (dollars, dollar_unit)
  elif cents:
    cent_unit = 'cent' if cents == 1 else 'cents'
    return '%s %s' % (cents, cent_unit)
  else:
    return 'zero dollars'


def _expand_ordinal(m):
  return _inflect.number_to_words(m.group(0))


def _expand_number(m):
  num = int(m.group(0))
  if num > 1000 and num < 3000:
    if num == 2000:
      return 'two thousand'
    elif num > 2000 and num < 2010:
      return 'two thousand ' + _inflect.number_to_words(num % 100)
    elif num % 100 == 0:
      return _inflect.number_to_words(num // 100) + ' hundred'
    else:
      return _inflect.number_to_words(num, andword='', zero='oh', group=2).replace(', ', ' ')
  else:
    return _inflect.number_to_words(num, andword='')


def normalize_numbers(text):
  text = re.sub(_comma_number_re, _remove_commas, text)
  text = re.sub(_pounds_re, r'\1 pounds', text)
  text = re.sub(_dollars_re, _expand_dollars, text)
  text = re.sub(_decimal_number_re, _expand_decimal_point, text)
  text = re.sub(_ordinal_re, _expand_ordinal, text)
  text = re.sub(_number_re, _expand_number, text)
  return text



# Regular expression matching whitespace:
_whitespace_re = re.compile(r"\s+")

def lowercase(text):
    return text.lower()

def collapse_whitespace(text):
    return re.sub(_whitespace_re, " ", text)

def basic_indic_cleaners(text):
    """Basic pipeline that collapses whitespace without transliteration."""
    text = collapse_whitespace(text)
    return text


def english_cleaner(text):
    text = text.lower().replace('‘','\'').replace('’','\'')
    return text


def lowercase(text):
    return text.lower()

def convert_to_ascii(text):
    return unidecode(text)

def expand_numbers(text):
    return normalize_numbers(text)

def expand_abbreviations(text):
    for regex, replacement in _abbreviations:
        text = re.sub(regex, replacement, text)
    return text

_abbreviations = [(re.compile('\\b%s\\.' % x[0], re.IGNORECASE), x[1]) for x in [
  ('mrs', 'missus'),
  ('mr', 'mister'),
  ('dr', 'doctor'),
  ('st', 'saint'),
  ('co', 'company'),
  ('jr', 'junior'),
  ('maj', 'major'),
  ('gen', 'general'),
  ('drs', 'doctors'),
  ('rev', 'reverend'),
  ('lt', 'lieutenant'),
  ('hon', 'honorable'),
  ('sgt', 'sergeant'),
  ('capt', 'captain'),
  ('esq', 'esquire'),
  ('ltd', 'limited'),
  ('col', 'colonel'),
  ('ft', 'fort'),
  ('pvt', 'private'),
  ('rs', 'Rupees')
]]






def english_cleaners(text):
    
    text = convert_to_ascii(text)
    text = lowercase(text)
    text = expand_numbers(text)
    text = expand_abbreviations(text)
    text = collapse_whitespace(text)
    return text


# Regular expression matching text enclosed in curly braces:
_curly_re = re.compile(r'(.*?)\{(.+?)\}(.*)')


def get_arpabet(word, dictionary):
  word_arpabet = dictionary.lookup(word)
  if word_arpabet is not None:
    return "{" + word_arpabet[0] + "}"
  else:
    return word


def text_to_sequence(text, symbols, cleaner_names, dictionary=None):
  '''Converts a string of text to a sequence of IDs corresponding to the symbols in the text.
    The text can optionally have ARPAbet sequences enclosed in curly braces embedded
    in it. For example, "Turn left on {HH AW1 S S T AH0 N} Street."
    Args:
      text: string to convert to a sequence
      cleaner_names: names of the cleaner functions to run the text through
      dictionary: arpabet class with arpabet dictionary
    Returns:
      List of integers corresponding to the symbols in the text
  '''
  # Mappings from symbol to numeric ID and vice versa:
  global _id_to_symbol, _symbol_to_id
  _symbol_to_id = {s: i for i, s in enumerate(symbols)}
  _id_to_symbol = {i: s for i, s in enumerate(symbols)}

  sequence = []

  space = _symbols_to_sequence(' ')
  # Check for curly braces and treat their contents as ARPAbet:
  while len(text):
    m = _curly_re.match(text)
    if not m:
      clean_text = _clean_text(text, cleaner_names)
      if dictionary is not None:
        clean_text = [get_arpabet(w, dictionary) for w in clean_text.split(" ")]
        for i in range(len(clean_text)):
          t = clean_text[i]
          if t.startswith("{"):
            sequence += _arpabet_to_sequence(t[1:-1])
          else:
            sequence += _symbols_to_sequence(t)
          sequence += space
      else:
        sequence += _symbols_to_sequence(clean_text)
      break
    sequence += _symbols_to_sequence(_clean_text(m.group(1), cleaner_names))
    sequence += _arpabet_to_sequence(m.group(2))
    text = m.group(3)
  
  # remove trailing space
  if dictionary is not None:
    sequence = sequence[:-1] if sequence[-1] == space[0] else sequence
  return sequence


def _clean_text(text, cleaner_names):
  for name in cleaner_names:
    #cleaner = getattr(cleaners, name)
    cleaner = name
    # print(cleaner)
    if not cleaner:
      raise Exception('Unknown cleaner: %s' % name)
    text = basic_indic_cleaners(text)
  return text


def _symbols_to_sequence(symbols):
  return [_symbol_to_id[s] for s in symbols if _should_keep_symbol(s)]


def _arpabet_to_sequence(text):
  return _symbols_to_sequence(['@' + s for s in text.split()])


def _should_keep_symbol(s):
  return s in _symbol_to_id and s is not '_' and s is not '~'



def check_directory(dir):
    if not os.path.exists(dir):
        sys.exit("Error: {} directory does not exist".format(dir))


class TextToMel:
    def __init__(self, glow_model_dir, device="cuda"):
        self.glow_model_dir = glow_model_dir
        check_directory(self.glow_model_dir)
        self.device = device
        self.hps, self.glow_tts_model = self.load_glow_tts()
        pass

    def load_glow_tts(self):
        hps = get_hparams_from_dir(self.glow_model_dir)
        checkpoint_path = latest_checkpoint_path(self.glow_model_dir)
        symbols = list(hps.data.punc) + list(hps.data.chars)
        glow_tts_model = models.FlowGenerator(
            len(symbols) + getattr(hps.data, "add_blank", False),
            out_channels=hps.data.n_mel_channels,
            **hps.model
        )  # .to(self.device)

        if self.device == "cuda":
            glow_tts_model.to("cuda")

        load_checkpoint(checkpoint_path, glow_tts_model)
        glow_tts_model.decoder.store_inverse()
        _ = glow_tts_model.eval()

        return hps, glow_tts_model

    def generate_mel(self, text, noise_scale=0.667, length_scale=1.0):
        symbols = list(self.hps.data.punc) + list(self.hps.data.chars)
        cleaner = self.hps.data.text_cleaners
        if getattr(self.hps.data, "add_blank", False):
            text_norm = text_to_sequence(text, symbols, cleaner)
            text_norm = commons.intersperse(text_norm, len(symbols))
        else:  # If not using "add_blank" option during training, adding spaces at the beginning and the end of utterance improves quality
            text = " " + text.strip() + " "
            text_norm = text_to_sequence(text, symbols, cleaner)

        sequence = np.array(text_norm)[None, :]

        del symbols
        del cleaner
        del text
        del text_norm

        if self.device == "cuda":
            x_tst = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
            x_tst_lengths = torch.tensor([x_tst.shape[1]]).cuda()
        else:
            x_tst = torch.autograd.Variable(torch.from_numpy(sequence)).long()
            x_tst_lengths = torch.tensor([x_tst.shape[1]])

        with torch.no_grad():
            (y_gen_tst, *_), *_, (attn_gen, *_) = self.glow_tts_model(
                x_tst,
                x_tst_lengths,
                gen=True,
                noise_scale=noise_scale,
                length_scale=length_scale,
            )
        del x_tst
        del x_tst_lengths
        torch.cuda.empty_cache()
        return y_gen_tst
        #return y_gen_tst.cpu().detach().numpy()


class MelToWav:
    def __init__(self, hifi_model_dir, device="cuda"):
        self.hifi_model_dir = hifi_model_dir
        check_directory(self.hifi_model_dir)
        self.device = device
        self.h, self.hifi_gan_generator = self.load_hifi_gan()
        pass

    def load_hifi_gan(self):
        checkpoint_path = latest_checkpoint_path(self.hifi_model_dir, regex="g_*")
        config_file = os.path.join(self.hifi_model_dir, "config.json")
        data = open(config_file).read()
        json_config = json.loads(data)
        h = AttrDict(json_config)
        torch.manual_seed(h.seed)

        generator = Generator(h).to(self.device)

        assert os.path.isfile(checkpoint_path)
        print("Loading '{}'".format(checkpoint_path))
        state_dict_g = torch.load(checkpoint_path, map_location=self.device)
        print("Complete.")

        generator.load_state_dict(state_dict_g["generator"])

        generator.eval()
        generator.remove_weight_norm()

        return h, generator

    def generate_wav(self, mel):
        #mel = torch.FloatTensor(mel).to(self.device)

        y_g_hat = self.hifi_gan_generator(mel.to(self.device))  # passing through vocoder
        audio = y_g_hat.squeeze()
        audio = audio * 32768.0
        audio = audio.cpu().detach().numpy().astype("int16")

        del y_g_hat
        del mel
        torch.cuda.empty_cache()
        return audio, self.h.sampling_rate



In [ ]:
# os.chdir('/content/vakyansh-tts/')

sys.path.append('/content/vakyansh-tts')
import bangla
# from tts_infer.tts import TextToMel, MelToWav
from tts_infer.transliterate import XlitEngine
from tts_infer.num_to_word_on_sent import normalize_nums

import re
from scipy.io.wavfile import write

text_to_mel = TextToMel(glow_model_dir='/content/vakyansh-tts/tts_infer/translit_models/bangla/glow_ckp', device=device)
mel_to_wav = MelToWav(hifi_model_dir='/content/vakyansh-tts/tts_infer/translit_models/bangla/hifi_ckp', device=device)

def translit(text, lang):
    reg = re.compile(r'[a-zA-Z]')
    engine = XlitEngine(lang)
    words = [engine.translit_word(word, topk=1)[lang][0] if reg.match(word) else word for word in text.split()]
    updated_sent = ' '.join(words)
    return updated_sent
    
def run_tts(text, lang):
    # text = text.replace('।', '.') # only for hindi models
    text_num_to_word = normalize_nums(text, lang) # converting numbers to words in lang
    text_num_to_word_and_transliterated = translit(text_num_to_word, lang) # transliterating english words to lang
    
    mel = text_to_mel.generate_mel(text_num_to_word_and_transliterated)
    audio, sr = mel_to_wav.generate_wav(mel)
    write(filename='temp.wav', rate=sr, data=audio) # for saving wav file, if needed
    return (sr, audio)

def ben_tts(text, lang):
    # text = text.replace('।', '.') # only for hindi models
    text_num_to_word = normalize_nums(text, lang) # converting numbers to words in lang
    text_num_to_word_and_transliterated = translit(text_num_to_word, lang) # transliterating english words to lang
    
    mel = text_to_mel.generate_mel(text_num_to_word_and_transliterated)
    audio, sr = mel_to_wav.generate_wav(mel)
    torch.cuda.empty_cache()
    gc.collect()
    return audio

/content/vakyansh-tts/tts_infer/translit_models/bangla/glow_ckp/G_200.pth


/content/vakyansh-tts/src/glow_tts/modules.py:234: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2491.)
  w_init = torch.qr(torch.FloatTensor(self.n_split, self.n_split).normal_())[0]


/content/vakyansh-tts/tts_infer/translit_models/bangla/hifi_ckp/g_00100000
Loading '/content/vakyansh-tts/tts_infer/translit_models/bangla/hifi_ckp/g_00100000'
Complete.
Removing weight norm...


In [ ]:
!pwd

/content/vakyansh-tts/tts_infer/translit_models/bangla


In [ ]:
os.chdir('/content/') 

In [ ]:
!pwd

/content


In [ ]:
_, audio = run_tts('বিসমিল্লাহ’র পূর্বে ‘আক্বরাউ’ ‘আবদাউ’ অথবা ‘আতলু’ ফে’ল ক্রিয়া উহ্য আছে। অর্থাৎ, আল্লাহর নাম নিয়ে পড়ছি  কিংবা তেলাঅত আরম্ভ করছি। ', 'bn')
ipd.Audio('temp.wav')

MB100% (39.0 of 39.0) |##################| Elapsed Time: 0:00:01 Time:  0:00:01


Succefully Downloaded to: /content/vakyansh-tts/tts_infer/translit_models/bengali.zip


/content/vakyansh-tts/src/glow_tts/models.py:399: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  y_lengths = (y_lengths // self.n_sqz) * self.n_sqz


In [ ]:
!nvidia-smi

Wed Oct 26 18:25:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    33W /  70W |   1202MiB / 15109MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Arabic Tacotron2 TTS

In [ ]:
!pip install pyarabic
from pyarabic.araby import strip_diacritics

from IPython.display import Audio
import soundfile as sf
import shutil


url = "https://drive.google.com/drive/folders/196xZbqqxzsBQdKr1UKdh_wL_9qtJkt6u?usp=sharing"
gdown.download_folder(url=url, quiet=True)  

shutil.copy('/content/arabic_tts/UNIVERSAL_V1/config.json', './tts-arabic-tacotron2/pretrained/hifigan-universal-v1')

shutil.copy('/content/arabic_tts/UNIVERSAL_V1/do_02500000', './tts-arabic-tacotron2/pretrained/hifigan-universal-v1')

shutil.copy('/content/arabic_tts/UNIVERSAL_V1/g_02500000', './tts-arabic-tacotron2/pretrained/hifigan-universal-v1')

shutil.copy('/content/arabic_tts/tacotron2_ar.pth', './tts-arabic-tacotron2/pretrained')

model_sd_path = './tts-arabic-tacotron2/pretrained/tacotron2_ar.pth'
# vocoder
vocoder_state_path = './tts-arabic-tacotron2/pretrained/hifigan-universal-v1/g_02500000'

vocoder_config_path = './tts-arabic-tacotron2/pretrained/hifigan-universal-v1/config.json'


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 126 kB 8.0 MB/s 


In [ ]:

%%writefile /content/tts-arabic-tacotron2/utils/__init__.py
import sys
import yaml
try:
    from yaml import CLoader as Loader
except ImportError:
    from yaml import Loader


class DictConfig(object):
    """Creates a Config object from a dict 
       such that object attributes correspond to dict keys.    
    """

    def __init__(self, config_dict):
        self.__dict__.update(config_dict)

    def __str__(self):
        return '\n'.join(f"{key}: {val}" for key, val in self.__dict__.items())

    def __repr__(self):
        return self.__str__()


def get_custom_config(fname):
    with open(fname, 'r') as stream:
        config_dict = yaml.load(stream, Loader)
    config = DictConfig(config_dict)
    return config


def get_basic_config():
    return get_custom_config('./tts-arabic-tacotron2/configs/basic.yaml')


def get_config(fname):
    config = get_basic_config()
    custom_config = get_custom_config(fname)

    config.__dict__.update(custom_config.__dict__)
    return config


def read_lines_from_file(path, encoding='utf-8'):
    lines = []
    with open(path, 'r', encoding=encoding) as f:
        for line in f:
            lines.append(line.strip())
    return lines


def progbar(iterable, length=30, symbol='='):
    """Wrapper generator function for an iterable. 
       Prints a progressbar when yielding an item. \\
       Args:
          iterable: an object supporting iteration
          length: length of the progressbar
    """
    n = len(iterable)
    for i, item in enumerate(iterable):
        steps = length*(i+1) // n
        sys.stdout.write('\r')
        sys.stdout.write(f"[{symbol*steps:{length}}] {(100/n*(i+1)):.1f}%")
        if i == (n-1):
            sys.stdout.write('\n')
        sys.stdout.flush()
        yield item



Overwriting /content/tts-arabic-tacotron2/utils/__init__.py


In [ ]:
ar_model = Tacotron2Wave(model_sd_path = model_sd_path,vocoder_sd = vocoder_state_path, vocoder_config = vocoder_config_path)
#ar_model = ar_model.cuda()
ar_model = ar_model.to(device) 

sample ='كَلاَّ بَلْ رَانَ عَلَى قُلُوبِهِمْ مَا كَانُوا يَكْسِبُونَ'

sample = strip_diacritics(sample)
print(sample)

wave = ar_model.tts(text_buckw = sample)
# wave = wave * 32768.0

كلا بل ران على قلوبهم ما كانوا يكسبون


In [ ]:

sf.write('sound.wav', wave, 22050)
Audio('./sound.wav')

#TTS BigTextToAudio

long text to audio conversion 

In [ ]:
import re
import torchaudio.functional as F
import torchaudio
from bnnumerizer import numerize
import gc
from bnunicodenormalizer import Normalizer 
# initialize
bnorm=Normalizer()
#loading empty audio file of  1 second to append before and after each arabic chunk for increasing mlt reading rhythm.

empty_audio, rate_of_sample = torchaudio.load('/content/empty.wav')
empty_audio = empty_audio.flatten()


def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None]) 


class BigTextToAudio(object):
    
    def __init__(self,
                 ar_model,
                 bn_model,
                 ar_sample_rate=22050,
                 bn_sample_rate=22050,
                 out_sample_rate=22050,
                 attribution_dict={"সাঃ":"সাল্লাল্লাহু আলাইহি ওয়া সাল্লাম",                   
                                  "আঃ":"আলাইহিস সালাম",
                                  "রাঃ":"রাদিআল্লাহু আনহু",
                                  "রহঃ":"রহমাতুল্লাহি আলাইহি",
                                  "রহিঃ":"রহিমাহুল্লাহ",
                                  "হাফিঃ":"হাফিযাহুল্লাহ",
                                  "বায়ান":"বাইআন",
                                  "দাঃবাঃ":"দামাত বারাকাতুহুম,দামাত বারাকাতুল্লাহ",
                                  "আয়াত" : "আইআত",#আইআত
                                  # "ওয়া" : "ওআ",
                                  # "ওয়াসাল্লাম"  : "ওআসাল্লাম",
                                  # "কেন"  : "কেনো",
                                  # "কোন" : "কোনো",
                                  # "বল"   : "বলো",
                                  # "চল"   : "চলো",
                                  # "কর"   : "করো",
                                  # "রাখ"   : "রাখো",
                                  "’"     :  "",
                                  "‘"     : "",
                                  # # "য়"     : "অ",
                                  # # "সম্প্রদায়" : "সম্প্রদাই",
                                  # "রয়েছে"   : "রইছে",
                                  # "রয়েছ"    : "রইছ",
                                   "/"   : " বাই ",
                                  },
                resample_params={"lowpass_filter_width": 64,
                                "rolloff": 0.9475937167399596,
                                "resampling_method": "kaiser_window",
                                "beta": 14.769656459379492}
                ):
        '''
            Instantiates a Big Text to Audio conversion object for bangla and arabic
            args:
                ar_model : arabic tts model
                bn_model : bangla tts model
                ar_sample_rate : arabic audio sample rate [optional] default: 22050
                bn_sample_rate : bangla audio sample rate [optional] default: 22050
                out_sample_rate : audio sample rate [optional] default: 22050
                attribution_dict : a dict for attribution expansion [optional]
                resample_params : audio resampling parameters [optional]
            resources:
                # Main class: modified from https://github.com/snakers4/silero-models/pull/174
                # Audio converter:https://www.kaggle.com/code/shahruk10/inference-notebook-wav2vec2
        '''
        self.ar_model = ar_model
        self.bn_model = bn_model
        self.attribution_dict=attribution_dict
 
        self.ar_sample_rate=ar_sample_rate
        self.bn_sample_rate=bn_sample_rate
        self.sample_rate=out_sample_rate  
        self.resample_params=resample_params
        
    # public
    def ar_tts(self,text):
        '''
            args: 
                text: arabic text (string)
            returns:
                audio as torch tensor
        '''

        text = strip_diacritics(text)
        try:
            audio = self.ar_model.tts(text)
            audio = torch.cat([empty_audio,audio], axis=0) #start empty
            audio = torch.cat([audio,empty_audio], axis=0) #end empty
            audio = audio * 32768.0
  
        except:
            print("failed ar => ",text)
            audio = empty_audio * 32768.0
                  
        return audio
    # public
    def bn_tts(self,text):
        '''
            args: 
                text   : bangla text (string)
            returns:
                audio as torch tensor
        '''
        if(self.bn_model == 'glowtts'):
          audio = ben_tts(text, 'bn')
          #audio = torch.from_numpy(audio)

        else:
          print("not supported!")

        return torch.tensor(audio)
    
    # public
    
    def expand_full_attribution(self,text):
        for word in self.attribution_dict:
            if word in text:
                text = text.replace(word, normalize(self.attribution_dict[word]))
        return text

    # def expand_full_attribution(self,text):
    #     for word,replacement in self.attribution_dict.items():
    #         text = re.sub(word,replacement,text)
    #     return normalize(text)
            
    def collapse_whitespace(self,text):
        # Regular expression matching whitespace:
        _whitespace_re = re.compile(r"\s+")
        return re.sub(_whitespace_re, " ", text)

    # public
    def tag_text(self,text):
        '''
            * tags arabic text with <ar>text</ar>
            * tags bangla text with <bn>text</bn>
        '''
        # remove multiple spaces
        text=re.sub(' +', ' ',text)
        # create start and end
        text="start"+text+"end"
        # tag text
        parts=re.split(u'[\u0600-\u06FF]+', text)
        # remove non chars
        parts=[p for p in parts if p.strip()]
        # unique parts
        parts=set(parts)
        # tag the text
        for m in parts:
            if len(m.strip())>1:text=text.replace(m,f"</ar><SPLIT><bn>{m}</bn><SPLIT><ar>")
        # clean-tags
        text=text.replace("</ar><SPLIT><bn>start",'<bn>')
        text=text.replace("end</bn><SPLIT><ar>",'</bn>')
        return text

    def process_text(self,text):
        '''
        process multilingual text for suitable MLT TTS format
            * expand attributions
            * numerize text
            * tag sections of the text
            * sequentially list text blocks
            * Split based on sentence ending Characters

        '''
        
        # english numbers to bangla conversion
        res = re.search('[0-9]', text)
        if res is not None:
          text = bangla.convert_english_digit_to_bangla_digit(text)
        
        #replace ':' in between two bangla numbers with ' এর '
        pattern=r"[০, ১, ২, ৩, ৪, ৫, ৬, ৭, ৮, ৯]:[০, ১, ২, ৩, ৪, ৫, ৬, ৭, ৮, ৯]"
        matches=re.findall(pattern,text)
        for m in matches:
            r=m.replace(":"," এর ")
            text=text.replace(m,r)

        # numerize text
        text=numerize(text)

        # tag sections
        text=self.tag_text(text)
        # text blocks
        blocks=text.split("<SPLIT>")
        blocks=[b for b in blocks if b.strip()]
        # create tuple of (lang,text)
        data=[]
        for block in blocks:
            lang=None
            if "<bn>" in block:
                block=block.replace("<bn>",'').replace("</bn>",'')
                lang="bn"
            elif "<ar>" in block:
                block=block.replace("<ar>",'').replace("</ar>",'')
                lang="ar"
            
            # Split based on sentence ending Characters

            if lang == "bn":
              bn_text = block.strip()

              sentenceEnders = re.compile('[।!?]')
              sentences = sentenceEnders.split(str(bn_text))

              for i in range(len(sentences)):
                  res = re.sub('\n','',sentences[i])
                  res = normalize(res)
                  # expand attributes
                  res=self.expand_full_attribution(res)

                  res = self.collapse_whitespace(res)
                  res += '।'
                  data.append((lang,res))

            elif lang == "ar":
                ar_text = block.strip()
                ar_text = re.sub("؟", "?", ar_text) # replace any ؟ with ?

                sentenceEnders = re.compile('[.!?]')
                sentences = sentenceEnders.split(str(ar_text))

                for i in range(len(sentences)):
                    res = re.sub('\n','',sentences[i])
                    res = self.collapse_whitespace(res)
                    data.append((lang,res))
                    
        return data
    
    def resample_audio(self,audio,sr):
        '''
            resample audio with sample rate
            args:
                audio : torch.tensor audio
                sr: audi sample rate
        '''
        if sr==self.sample_rate:
            return audio
        else:
            return F.resample(audio,sr,self.sample_rate,**self.resample_params)
        
    
    def get_audio(self,data):
        '''
            creates audio from given data 
                * data=List[Tuples(lang,text)]
        '''
        audio_list = []
        for block in data:
            lang,text=block
            if lang=="bn":
                audio=self.bn_tts(text)
                sr=self.bn_sample_rate
            else:
                audio=self.ar_tts(text)
                sr=self.ar_sample_rate
            
            if self.resample_audio_to_out_sample_rate:
                audio=self.resample_audio(audio,sr)
                
            audio_list.append(audio)
  
        audio = torch.cat([k for k in audio_list])
        return audio
    
    # call
    def __call__(self,text,resample_audio_to_out_sample_rate=True):
        '''
            args: 
                text   : bangla text (string)
                resample_audio_to_out_sample_rate: for different sample rate in different models, resample the output audio 
                                                   in uniform sample rate 
                                                   * default:True
            returns:
                audio as numpy data
        '''
        self.resample_audio_to_out_sample_rate=resample_audio_to_out_sample_rate
        data=self.process_text(text)
        audio=self.get_audio(data)
        return audio.detach().cpu().numpy()

#check textprocessing

In [ ]:

df = pd.read_csv('/content/bn_qtafsir/ben_quran_with_tafsir.csv')
text = "আয়াত টির বাংলা তাফসির পরছি। "+df.tafsir_bayan_text[1]
#print(text)
bn_model = 'glowtts'
MLT_TTS=BigTextToAudio(ar_model=ar_model,
                   bn_model=bn_model)

data=MLT_TTS.process_text(text)
data

[('bn', 'আইআত টির বাংলা তাফসির পরছি।'),
 ('bn', '[এক]।'),
 ('ar', 'الحَمد'),
 ('bn', 'এর মধ্যে যে।'),
 ('ar', 'ال'),
 ('bn', 'রয়েছে, তা।'),
 ('ar', 'استغراق'),
 ('bn', '(সমূদয়) অথবা।'),
 ('ar', 'اختصاص'),
 ('bn', '(নির্দিষ্টীকরণ)এর অর্থে ব্যবহার হয়েছে।'),
 ('bn',
  'অর্থাৎ, সমস্ত প্রশংসা আল্লাহর জন্যই বা তাঁর জন্য নির্দিষ্ট; কেননা প্রশংসার প্রকৃত অধিকারী একমাত্র মহান আল্লাহই।'),
 ('bn',
  'কারো মধ্যে যদি কোন গুণ, সৌন্দর্য এবং কৃতিত্ব থাকে, তবে তাও মহান আল্লাহ কর্তৃক সৃষ্ট।'),
 ('bn', 'অতএব প্রশংসার অধিকারী তিনিই।'),
 ('bn',
  'আল্লাহ শব্দটি মহান আল্লাহর সত্তার এমন এক সতন্ত্র নাম যার ব্যবহার অন্য কারো জন্য করা বৈধ নয়।'),
 ('bn', 'আলহামদু লিল্লাহ কৃতজ্ঞতা-জ্ঞাপক বাক্য।'),
 ('bn', 'এর বহু ফযীলতের কথা হাদীসসমূহে এসেছে।'),
 ('bn',
  'একটি হাদীসে লা-ইলাহা ইল্লাল্লা-হকে উত্তম জিকির বলা হয়েছে এবং আলহামদু লিল্লাহকে উত্তম দুআ বলা হয়েছে।'),
 ('bn',
  '(তিরমিযী, নাসায়ী ইত্যাদি) সহীহ মুসলিম এবং নাসায়ীর বর্ণনায় এসেছে, আলহামদু লিল্লাহ দাঁড়িপাল্লা ভর্তি করে দেয়।'),
 ('bn',
  'এ জন্যই অন্য এক বর্ণনায় এ

In [ ]:

audio=MLT_TTS(text,resample_audio_to_out_sample_rate=True)
Audio(audio, rate=MLT_TTS.sample_rate)

In [ ]:
sf.write(f'./check_mlt.wav', audio/ 32768.0, 22050)
# Audio('/content/check_mlt.wav', autoplay=True)

In [ ]:
len(df.tafsir_bayan_text)

6236

In [1]:
def mlt_reader(surah_list,df):
    '''
      surah_list -> list of surah's for what we will be launching MLT TTS
      df         -> dataframe containing all bangla quranic verses and tafsir
    '''
    surah_curr=surah_list[0]
    bn_audio = []
    for i in range(len(df)):
        surah = df.ayat[i].split(",")[0]
        surah_num = int(re.findall(r'\d+', surah)[0])
        
        if(surah_num not in surah_list):
            continue
        else:
            bn_verse = df.ayat[i].replace("surah","সুরা").replace("ayat","আয়াত")
            bn_verse = bangla.convert_english_digit_to_bangla_digit(bn_verse)
            bn_qtafsir = bn_verse +" এর বাংলা অনুবাদ পরছি। "+ df.আল_বায়ান[i] + " । " + " আয়াত টির বাংলা তাফসির পরছি। "+df.tafsir_bayan_text[i] 
            audio=MLT_TTS(bn_qtafsir,resample_audio_to_out_sample_rate=True)
            # if current surah is bein
            if surah_curr==surah_num:    
                bn_audio.append(audio)

                full_surah_audio=np.concatenate(bn_audio,axis=-1)
                sf.write(f'/content/drive/MyDrive/Books/{surah_num}.wav', full_surah_audio/ 32768.0, 22050)
            else:
                bn_audio=[]
                bn_audio.append(audio)
                surah_curr=surah_num

In [ ]:
memory=13109#/240 #(GPU total memory/number of cores)
os.environ["PYTORCH_CUDA_ALLOC_CONF"]="max_split_size_mb:{}".format(memory)

In [ ]:
%%time
df = df.replace(np.nan, 'এটির তাফসির নেই।', regex=True)
surah_list = [2]
mlt_reader(surah_list = surah_list,df = df)

CPU times: user 1h 5min 3s, sys: 17min 22s, total: 1h 22min 25s
Wall time: 1h 43min 18s


In [ ]:
!pwd

/content


In [ ]:
# from IPython.display import Audio 
# Audio('/content/1.wav', autoplay=True)

!nvidia-smi

Wed Oct 26 20:09:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    30W /  70W |   1402MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------